In [1]:
# Import Libraries

import numpy as np
import pandas as pd


In [2]:
# Load dataset

dir = "./heart+disease/processed.switzerland.data"

columns = [
  'age', 'sex', 'cp', 'trestbps', 'chol',
  'fbs', 'restecg', 'thalach', 'exang', 'oldpeak',
  'slope', 'ca', 'thal', 'num'
]

# Read dataset
df = pd.read_csv(dir, names=columns, header=None)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,32,1,1,95,0,?,0,127,0,.7,1,?,?,1
1,34,1,4,115,0,?,?,154,0,.2,1,?,?,1
2,35,1,4,?,0,?,0,130,1,?,?,?,7,3
3,36,1,4,110,0,?,0,125,1,1,2,?,6,1
4,38,0,4,105,0,?,0,166,0,2.8,1,?,?,2


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   age       123 non-null    int64 
 1   sex       123 non-null    int64 
 2   cp        123 non-null    int64 
 3   trestbps  123 non-null    object
 4   chol      123 non-null    int64 
 5   fbs       123 non-null    object
 6   restecg   123 non-null    object
 7   thalach   123 non-null    object
 8   exang     123 non-null    object
 9   oldpeak   123 non-null    object
 10  slope     123 non-null    object
 11  ca        123 non-null    object
 12  thal      123 non-null    object
 13  num       123 non-null    int64 
dtypes: int64(5), object(9)
memory usage: 13.6+ KB


In [4]:
# Melihat data unique
cols = df.columns

for col in cols:
    print(f"Kolom {col}")
    print(df[col].unique())
    print("\n")

Kolom age
[32 34 35 36 38 40 41 42 43 45 46 47 48 50 51 52 53 54 55 56 57 58 59 60
 61 62 63 64 65 66 67 68 69 70 72 73 74]


Kolom sex
[1 0]


Kolom cp
[1 4 3 2]


Kolom trestbps
['95' '115' '?' '110' '105' '100' '135' '150' '125' '145' '140' '155'
 '160' '120' '130' '165' '80' '180' '170' '200' '185']


Kolom chol
[0]


Kolom fbs
['?' '0' '1']


Kolom restecg
['0' '?' '1' '2']


Kolom thalach
['127' '154' '130' '125' '166' '156' '179' '128' '150' '120' '144' '176'
 '99' '122' '145' '140' '138' '133' '113' '118' '149' '124' '110' '139'
 '92' '104' '170' '163' '60' '126' '82' '95' '115' '135' '141' '155' '83'
 '97' '98' '100' '148' '103' '121' '131' '182' '105' '175' '94' '119'
 '143' '63' '70' '77' '117' '123' '134' '72' '78' '109' '86' '114' '93'
 '67' '90' '108' '136' '?' '157']


Kolom exang
['0' '1' '?']


Kolom oldpeak
['.7' '.2' '?' '1' '2.8' '0' '-1.1' '1.6' '-1.5' '1.5' '2' '.5' '-.1'
 '-2.6' '2.1' '-.7' '2.2' '3' '.1' '.3' '-2' '-1' '1.8' '1.4' '2.6' '.9'
 '2.4' '1.1' '.4' '2

In [5]:
df.replace('?', np.nan, inplace=True)

In [6]:
df.isnull().sum()

age           0
sex           0
cp            0
trestbps      2
chol          0
fbs          75
restecg       1
thalach       1
exang         1
oldpeak       6
slope        17
ca          118
thal         52
num           0
dtype: int64

In [ ]:
numMissVal = 